In [1]:
import requests

In [ ]:
# a request to localhost:8080

url = "http://localhost:8080"
# url = "https://mypersonaljscloudivnad9dy-readnft.functions.fnc.fr-par.scw.cloud"

#my_prompt = [
#    {"role": "system", "content": "You are a helpful assistant."},
#    {"role": "user", "content": "Where is Chile?"}
# ]
my_prompt = "Where is Chile?"  # Example prompt, can be replaced with any string

# send a get request with the prompt and tokenId as query parameters
response = requests.get(
    url, params={"prompt": my_prompt}
)